In [1]:
import pandas as pd
import numpy as np

In [32]:
close_returns_raw = pd.read_csv('dataset/close_returns.csv', index_col = 0, parse_dates = True)
high_prices_raw = pd.read_csv('dataset/high_prices.csv', index_col = 0, parse_dates = True)
low_prices_raw = pd.read_csv('dataset/low_prices.csv', index_col = 0, parse_dates = True)
volume_raw = pd.read_csv('dataset/volume.csv', index_col = 0, parse_dates = True)
signals_A_raw = pd.read_csv('dataset/signals_A.csv', index_col = 0, parse_dates = True)
signals_B_raw = pd.read_csv('dataset/signals_B.csv', index_col = 0, parse_dates = True)

In [33]:
close_returns_raw.head()

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.002627,-0.005865,-0.002815,-0.006875,0.004111,-0.007562,0.034228,-0.016570,-0.023604,...,0.002529,-0.005074,0.000942,0.005411,NaN,-0.002108,0.006905,-0.004139,0.006602,0.002468
2017-01-03 10:00:00,-0.005218,-0.003813,0.006697,-0.006235,-0.001562,0.030088,0.017317,-0.014377,0.002730,-0.003007,...,-0.001575,NaN,0.019034,-0.003286,NaN,-0.009330,NaN,0.004126,0.001973,0.023038
2017-01-03 10:15:00,0.004456,0.007341,0.006147,-0.001772,-0.001146,-0.018533,-0.015283,0.028084,-0.015930,0.001107,...,0.001118,NaN,-0.003259,-0.004713,NaN,-0.000681,NaN,0.008012,0.006369,-0.019062
2017-01-03 10:30:00,-0.001799,-0.010040,0.008337,-0.015490,0.000895,-0.022239,0.011906,NaN,0.034934,-0.008240,...,0.005739,NaN,-0.002320,-0.003620,NaN,-0.001107,0.007323,-0.018825,-0.004333,0.000031


In [16]:
#def train_test_split_and_fillna(dataframe):
#    # Fill forward 
#    train_data = dataframe[:4360]
#    test_data = dataframe[4360:]   

'2017-08-31 15:45:00'

In [35]:
median_volume_each_stock = volume_raw.median(axis = 0)
volume_normalised = volume_raw.apply(lambda x: x / median_volume_each_stock, axis=1)

In [36]:
volatility = (high_prices_raw - low_prices_raw) / low_prices_raw

In [72]:
#volume_normalised_filled = volume_normalised.groupby(
#    volume_normalised.index.dayofyear).apply(
#    lambda x: x.fillna(method = "ffill", axis = 0, limit = 4))

In [75]:
def intra_day_forward_fillna_max_filling_1_hour_gap(dataframe):
    return dataframe.groupby(dataframe.index.dayofyear).apply(
        lambda x: x.fillna(method = "ffill", axis = 0, limit = 4))

In [76]:
volume_normalised_filled = intra_day_forward_fillna_max_filling_1_hour_gap(volume_normalised)
volatility_filled = intra_day_forward_fillna_max_filling_1_hour_gap(volatility)
signal_A_raw_filled = intra_day_forward_fillna_max_filling_1_hour_gap(signals_A_raw)
signal_B_raw_filled = intra_day_forward_fillna_max_filling_1_hour_gap(signals_B_raw)

In [82]:
close_returns_raw_filled = close_returns_raw.groupby(close_returns_raw.index.dayofyear).apply(
    lambda x: x.fillna(value = 0.0, axis = 0)) 
# Different from "ffill", if use "limit" here, it would specify maximum 
# total number of 0 that can be used to pad each col. Not what we want.

In [88]:
type(close_returns_raw_filled.iloc[:,1])

pandas.core.series.Series

In [101]:
list_of_df_each_for_one_stock = [pd.concat([close_returns_raw_filled.iloc[:, a],
                                            volume_normalised_filled.iloc[:, a],
                                            volatility_filled.iloc[:, a],
                                            signal_A_raw_filled.iloc[:, a],
                                            signal_B_raw_filled.iloc[:, a]], axis = 1)
                                 for a in range(1000)] # Total 1000 stocks

In [109]:
for e in list_of_df_each_for_one_stock:
    e.columns = ["return", "volume", "volatility", "signal_A", "signal_B"]               

In [83]:
close_returns_raw_filled

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-03 09:45:00,-0.010530,0.002627,-0.005865,-0.002815,-0.006875,0.004111,-0.007562,0.034228,-0.016570,-0.023604,...,0.002529,-0.005074,0.000942,0.005411,0.0,-0.002108,0.006905,-0.004139,0.006602,0.002468
2017-01-03 10:00:00,-0.005218,-0.003813,0.006697,-0.006235,-0.001562,0.030088,0.017317,-0.014377,0.002730,-0.003007,...,-0.001575,0.000000,0.019034,-0.003286,0.0,-0.009330,0.000000,0.004126,0.001973,0.023038
2017-01-03 10:15:00,0.004456,0.007341,0.006147,-0.001772,-0.001146,-0.018533,-0.015283,0.028084,-0.015930,0.001107,...,0.001118,0.000000,-0.003259,-0.004713,0.0,-0.000681,0.000000,0.008012,0.006369,-0.019062
2017-01-03 10:30:00,-0.001799,-0.010040,0.008337,-0.015490,0.000895,-0.022239,0.011906,0.000000,0.034934,-0.008240,...,0.005739,0.000000,-0.002320,-0.003620,0.0,-0.001107,0.007323,-0.018825,-0.004333,0.000031
2017-01-03 10:45:00,-0.001767,0.002258,-0.010458,-0.011157,0.005186,0.018915,-0.013479,0.000000,-0.024356,0.008626,...,-0.014233,-0.009030,-0.006441,-0.004443,0.0,-0.008716,-0.007838,-0.000796,0.002679,-0.029635
2017-01-03 11:00:00,0.000046,0.011419,0.002151,0.009836,0.010735,-0.008125,0.005566,0.000000,0.004592,0.006912,...,0.012062,0.000000,-0.004638,0.018992,0.0,-0.006993,0.000000,0.004060,-0.009044,0.003350
2017-01-03 11:15:00,0.004687,-0.001928,-0.001605,-0.003722,-0.019821,0.007493,-0.013637,0.000000,-0.000252,-0.018014,...,0.001177,0.000000,0.011449,-0.006841,0.0,0.009667,0.000000,0.004176,0.006491,0.011093
2017-01-03 11:30:00,-0.003899,-0.005970,0.003769,-0.002684,0.010164,-0.012459,-0.015187,0.000000,-0.007469,-0.008912,...,-0.006383,0.000000,-0.015851,-0.000594,0.0,-0.003733,0.000000,-0.002952,-0.008927,-0.011230


In [80]:
close_returns_raw

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.002627,-0.005865,-0.002815,-0.006875,0.004111,-0.007562,0.034228,-0.016570,-0.023604,...,0.002529,-0.005074,0.000942,0.005411,NaN,-0.002108,0.006905,-0.004139,0.006602,0.002468
2017-01-03 10:00:00,-0.005218,-0.003813,0.006697,-0.006235,-0.001562,0.030088,0.017317,-0.014377,0.002730,-0.003007,...,-0.001575,NaN,0.019034,-0.003286,NaN,-0.009330,NaN,0.004126,0.001973,0.023038
2017-01-03 10:15:00,0.004456,0.007341,0.006147,-0.001772,-0.001146,-0.018533,-0.015283,0.028084,-0.015930,0.001107,...,0.001118,NaN,-0.003259,-0.004713,NaN,-0.000681,NaN,0.008012,0.006369,-0.019062
2017-01-03 10:30:00,-0.001799,-0.010040,0.008337,-0.015490,0.000895,-0.022239,0.011906,NaN,0.034934,-0.008240,...,0.005739,NaN,-0.002320,-0.003620,NaN,-0.001107,0.007323,-0.018825,-0.004333,0.000031
2017-01-03 10:45:00,-0.001767,0.002258,-0.010458,-0.011157,0.005186,0.018915,-0.013479,NaN,-0.024356,0.008626,...,-0.014233,-0.009030,-0.006441,-0.004443,NaN,-0.008716,-0.007838,-0.000796,0.002679,-0.029635
2017-01-03 11:00:00,0.000046,0.011419,0.002151,0.009836,0.010735,-0.008125,0.005566,NaN,0.004592,0.006912,...,0.012062,NaN,-0.004638,0.018992,NaN,-0.006993,NaN,0.004060,-0.009044,0.003350
2017-01-03 11:15:00,0.004687,-0.001928,-0.001605,-0.003722,-0.019821,0.007493,-0.013637,NaN,-0.000252,-0.018014,...,0.001177,NaN,0.011449,-0.006841,NaN,0.009667,NaN,0.004176,0.006491,0.011093
2017-01-03 11:30:00,-0.003899,-0.005970,0.003769,-0.002684,0.010164,-0.012459,-0.015187,NaN,-0.007469,-0.008912,...,-0.006383,NaN,-0.015851,-0.000594,NaN,-0.003733,NaN,-0.002952,-0.008927,-0.011230


In [74]:
volume_normalised_filled

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,1.357828,2.131655,0.280324,0.605859,3.425711,2.090937,2.245844,0.243041,1.073698,2.198757,...,6.525990,11.020202,2.127320,1.674514,NaN,3.957813,11.325221,2.862198,1.743123,2.763214
2017-01-03 09:45:00,0.840949,1.523605,0.095504,1.379818,3.601746,2.252931,2.352143,0.056230,1.330452,3.479407,...,1.214361,0.255051,1.314881,0.991618,NaN,1.861504,0.111726,1.980825,1.060143,0.333148
2017-01-03 10:00:00,1.191392,4.844305,0.165257,0.452865,3.037720,5.138594,2.660210,0.153559,2.754052,3.377359,...,2.829630,0.255051,1.643113,1.575263,NaN,3.447960,0.111726,1.982614,1.320626,0.480422
2017-01-03 10:15:00,0.554745,1.383915,0.144401,0.300260,2.464698,1.759046,1.257130,0.099944,0.300303,0.794931,...,1.152344,0.255051,2.265060,1.068040,NaN,2.049958,1.424779,2.212153,0.761715,0.459780
2017-01-03 10:30:00,0.484170,5.054845,0.229429,0.675521,1.973222,1.735544,0.890956,0.099944,0.174951,2.151623,...,1.113297,0.252525,0.921198,0.884609,NaN,1.891479,0.221239,1.255255,0.929012,0.224938
2017-01-03 10:45:00,0.742527,0.803121,0.277288,0.754427,1.794610,1.278798,0.521275,0.099944,0.272423,1.970730,...,0.920607,0.502525,2.737870,1.013019,NaN,0.535665,0.768805,1.860130,0.747822,1.107655
2017-01-03 11:00:00,1.119380,0.950311,0.113187,0.992708,2.368259,0.535957,1.317218,0.099944,0.232656,1.100812,...,1.520797,0.502525,2.806578,1.209738,NaN,2.564530,0.768805,1.721210,0.858380,0.683699
2017-01-03 11:15:00,0.694956,0.998259,0.737007,0.323177,2.083440,0.435056,1.983834,0.089856,0.229522,0.803698,...,1.888650,0.502525,1.242592,0.990993,NaN,1.889259,0.222345,1.264647,0.692759,1.234194
2017-01-03 11:30:00,0.841778,0.777875,0.132143,0.298047,1.931746,0.817422,2.486961,0.089856,0.630106,3.247623,...,2.035252,28.588384,1.094432,0.815766,NaN,0.821538,0.222345,1.630479,0.492749,0.511895


In [73]:
volume_normalised

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,1.357828,2.131655,0.280324,0.605859,3.425711,2.090937,2.245844,0.243041,1.073698,2.198757,...,6.525990,11.020202,2.127320,1.674514,NaN,3.957813,11.325221,2.862198,1.743123,2.763214
2017-01-03 09:45:00,0.840949,1.523605,0.095504,1.379818,3.601746,2.252931,2.352143,0.056230,1.330452,3.479407,...,1.214361,0.255051,1.314881,0.991618,NaN,1.861504,0.111726,1.980825,1.060143,0.333148
2017-01-03 10:00:00,1.191392,4.844305,0.165257,0.452865,3.037720,5.138594,2.660210,0.153559,2.754052,3.377359,...,2.829630,NaN,1.643113,1.575263,NaN,3.447960,NaN,1.982614,1.320626,0.480422
2017-01-03 10:15:00,0.554745,1.383915,0.144401,0.300260,2.464698,1.759046,1.257130,0.099944,0.300303,0.794931,...,1.152344,NaN,2.265060,1.068040,NaN,2.049958,1.424779,2.212153,0.761715,0.459780
2017-01-03 10:30:00,0.484170,5.054845,0.229429,0.675521,1.973222,1.735544,0.890956,NaN,0.174951,2.151623,...,1.113297,0.252525,0.921198,0.884609,NaN,1.891479,0.221239,1.255255,0.929012,0.224938
2017-01-03 10:45:00,0.742527,0.803121,0.277288,0.754427,1.794610,1.278798,0.521275,NaN,0.272423,1.970730,...,0.920607,0.502525,2.737870,1.013019,NaN,0.535665,0.768805,1.860130,0.747822,1.107655
2017-01-03 11:00:00,1.119380,0.950311,0.113187,0.992708,2.368259,0.535957,1.317218,NaN,0.232656,1.100812,...,1.520797,NaN,2.806578,1.209738,NaN,2.564530,NaN,1.721210,0.858380,0.683699
2017-01-03 11:15:00,0.694956,0.998259,0.737007,0.323177,2.083440,0.435056,1.983834,0.089856,0.229522,0.803698,...,1.888650,NaN,1.242592,0.990993,NaN,1.889259,0.222345,1.264647,0.692759,1.234194
2017-01-03 11:30:00,0.841778,0.777875,0.132143,0.298047,1.931746,0.817422,2.486961,NaN,0.630106,3.247623,...,2.035252,28.588384,1.094432,0.815766,NaN,0.821538,NaN,1.630479,0.492749,0.511895


In [45]:
dir(close_returns_raw.index[1])

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_date_attributes',
 '_date_repr',
 '_get_date_name_field',
 '_get_start_end_field',
 '_has_time_component',
 '_repr_base',
 '_round',
 '_short_repr',
 '_time_repr',
 'asm8',
 'astimezone',
 'ceil',
 'combine',
 'ctime',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'dst',
 'floor',
 'fold',
 'freq',
 'freqstr',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'isocalenda

In [59]:
close_returns_raw.index[90].date()

datetime.date(2017, 1, 6)

In [57]:
close_returns_raw.index[90].dayofyear

6